In [1]:
%load_ext autoreload
%autoreload 2
import os
import random
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.gridspec import GridSpec
#import seaborn as sns
import sys
sys.path.append('../../')
from dataset import Dataset, SpikingDataset, RegSpikingDataset
from torch.utils.data.dataloader import DataLoader
from Loss import KDLoss
import torch
import torch.nn.functional as F
import torch.nn as nn
import torchvision
random.seed(1338)
import scipy.io
import numpy as np
import matplotlib.pyplot as plt
import copy
from pyESN import ESN
from scipy import interpolate
from gen_data import *
from tanh import tanh
from DFRSystem import *
import pandas as pd

from Loss import ber_loss


import os
import h5py
import numpy as np
import matplotlib.pyplot as plt
import torch
from torch.utils.data import Dataset, DataLoader

# import slayer from lava-dl
import lava.lib.dl.slayer as slayer

import IPython.display as display
from matplotlib import animation

generating_data = False

%load_ext autoreload

%autoreload 2

torch.__version__
# The coarse network structure is dicated by the Fashion MNIST dataset. 
dtype = torch.float

# Check whether a GPU is available
if torch.cuda.is_available():
    device = torch.device('cuda:0')
    #device = torch.device("cuda")     
else:
    device = torch.device("cpu")
    
print(device)
tau_mem = 10e-3
tau_syn = 5e-3
time_step = 1e-3
alpha   = float(np.exp(-time_step/tau_syn))
beta    = float(np.exp(-time_step/tau_mem))

weight_scale = 7*(1.0-beta) # this should give us some spikes to begin with

print("init done")

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
cpu
init done


In [2]:
def pre_processing(train_input, train_label):
    idx_p = 10
    begin = 0 # N_total_frame * N_sync_frame
    
    # label index for data
    train_input_df = pd.DataFrame(train_input, columns = ['1','2', '3', '4'])
    #train_input_df['L1_idx'] = train_input_df.index % idx_p

    # label index for label
    train_label_df = pd.DataFrame(train_label, columns = ['L1','L2'])
    train_label_df['L1_idx'] = train_label_df.index % idx_p
    
    # split training and testing data
    test_input_df, test_label_df = train_input_df.iloc[75* 80 + 1:, :], train_label_df.iloc[75* 80 + 1:, :]
    train_input_df, train_label_df = train_input_df.iloc[:75* 80 + 1, :], train_label_df.iloc[:75* 80 + 1, :]

    # group by 
    #mapping = train_label_df.loc[begin:, :].groupby(by='L1_idx').mean().reset_index().loc[:, ['L1', 'L2', 'L1_idx']] 
    
    #train_input_df = pd.merge(train_input_df, mapping, how='left', on='L1_idx')

    #train_input_df = pd.get_dummies(train_input_df, prefix=['L'], columns=['L1_idx'])

    train_input_df = train_input_df.loc[begin:, :]


    #print(train_input_df.head())
    
    # testing data
    # group by
    #test_input_df = test_input_df.merge(mapping, how = 'left', on='L1_idx')

    #test_input_df = pd.get_dummies(test_input_df, prefix=['L'], columns=['L1_idx'])

    #print(test_input_df.head())

    train_input = train_input_df.to_numpy()
    test_input = test_input_df.to_numpy()
    
    train_label = train_label_df.drop(['L1_idx'], axis=1).to_numpy()
    test_label = test_label_df.drop(['L1_idx'], axis=1).to_numpy()
    
    #(train_input.shape)
    #print(test_input.shape)
    
    
    return train_input, train_label, test_input, test_label

In [3]:
class LSTMNet(nn.Module):
    def __init__(self, input_size, output_size):
        super().__init__()
        self.lstm = nn.LSTM(input_size=input_size, hidden_size=1, num_layers=1, batch_first=True, bias=False)
        self.rnn = nn.RNN(input_size=input_size, hidden_size=32, num_layers=1, batch_first=True, bias=False)
        #self.rnn.weight_ih_l0.requires_grad_(False)
        #self.rnn.weight_hh_l0.requires_grad_(False)
        #self.fc1 = nn.Linear(16, 8)
        self.fc1 = nn.Linear(32, 16)
        self.fc2 = nn.Linear(16, output_size)
        self.act = nn.ReLU()

    def forward(self, x):
        rnn_x, _ = self.rnn(x)
        rnn_x = rnn_x[:, -1, :]
        #x = x.flatten(start_dim=1)
        #x = torch.cat((x, rnn_x), axis=1)
        x = self.fc1(rnn_x)
        x = self.act(x)
        x = self.fc2(x)
        # x = self.act(x)
        # x = self.fc2(x)
        # x = self.act(x)
        # x = self.fc3(x)
        return x

class CNNNet(nn.Module):
    def __init__(self, in_ch, input_size, output_size):
        super().__init__()
        self.conv1 = torch.nn.Conv1d(in_channels=in_ch, out_channels=16, kernel_size=3, stride=1, padding=1)
        self.conv2 = torch.nn.Conv1d(in_channels=16, out_channels=16, kernel_size=3, stride=1, padding=1)
        self.fc1 = nn.Linear(16 * input_size, output_size)
        self.act = nn.ReLU()

    def forward(self, x):
        x = self.conv1(x)
        x = self.act(x)
        x = self.conv2(x)
        x = self.act(x)
        x = x.flatten(start_dim=1)
        x = self.fc1(x)
        return x
    
    
class MLPNet(nn.Module):
    def __init__(self, input_size, output_size):
        super().__init__()
        self.fc1 = nn.Linear(input_size, 16)
        self.fc2 = nn.Linear(16, 16)
        self.fc3 = nn.Linear(16, 2)
        self.act = nn.ReLU()

    def forward(self, x):
        x = self.fc1(x)
        x = self.act(x)
        x = self.fc2(x)
        x = self.act(x)
        x = self.fc3(x)
        return x

In [4]:
def test_rnn(test_loader, model, rc, nb_inputs, num_frame=19):
    all_output = []
    inputs = []
    labels = []
    
    for input, input1, input2, target in test_loader:
        inputs.append(input)
        labels.append(target.cpu().detach().numpy())
        target = target.float().to(device)
        input = input.float().to(device)
        input1 = input1.float().to(device)
        input2 = input2.float().to(device)
        
        input2 = input2.view(input2.shape[0], nb_inputs, 4)
        
        output = model(input2).cpu().detach().numpy()
        all_output.append(output)
        
    
    all_output = np.concatenate(all_output, axis=0)
    labels = np.concatenate(labels, axis=0)
   
    predict_time = rc.time_to_freq(all_output, num_frame, remove_delay=False)
    target_time = rc.time_to_freq(labels, num_frame, remove_delay=False)
    return rc.my_new_test(predict_time, target_time)

def test_cnn(test_loader, model, rc, nb_inputs, num_frame=19):
    all_output = []
    inputs = []
    labels = []
    
    for input, input1, input2, target in test_loader:
        inputs.append(input)
        labels.append(target.cpu().detach().numpy())
        target = target.float().to(device)
        input = input.float().to(device)
        input1 = input1.float().to(device)
        input2 = input2.float().to(device)
        
        input2 = input2.view(input2.shape[0], nb_inputs, 4)
        
        output = model(input2).cpu().detach().numpy()
        all_output.append(output)
        
    
    all_output = np.concatenate(all_output, axis=0)
    labels = np.concatenate(labels, axis=0)
   
    predict_time = rc.time_to_freq(all_output, num_frame, remove_delay=False)
    target_time = rc.time_to_freq(labels, num_frame, remove_delay=False)
    return rc.my_new_test(predict_time, target_time)


def test_mlp(test_loader, model, rc, nb_inputs, num_frame=19):
    all_output = []
    inputs = []
    labels = []
    
    for input, input1, input2, target in test_loader:
        inputs.append(input)
        labels.append(target.cpu().detach().numpy())
        target = target.float().to(device)
        input = input.float().to(device)
        input1 = input1.float().to(device)
        input2 = input2.float().to(device)
        
        input2 = input2.view(input2.shape[0], nb_inputs * 4)
        
        output = model(input2).cpu().detach().numpy()
        all_output.append(output)
        
    
    all_output = np.concatenate(all_output, axis=0)
    labels = np.concatenate(labels, axis=0)
   
    predict_time = rc.time_to_freq(all_output, num_frame, remove_delay=False)
    target_time = rc.time_to_freq(labels, num_frame, remove_delay=False)
    return rc.my_new_test(predict_time, target_time)


In [5]:
def train_dnn(trainloader, testloader, test_teacher, net, rc, nb_inputs, lr=2e-3, nb_epochs=10):
    optimizer = torch.optim.Adam(net.parameters(), lr=lr, betas=(0.9,0.999), weight_decay=1e-4)
    scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=30, gamma=0.1)
    
    loss_fn = torch.nn.L1Loss()
    loss_hist = []
    best_ber = 10000000000.0
    
    for e in range(nb_epochs):
        loss_hist = []
        for x1_local, x2_local, x3_local, y_local in trainloader:
            x1_local = x1_local.float().to(device)
            x2_local = x2_local.float().to(device)
            x3_local = x3_local.float().to(device)
            y_local = y_local.float().to(device)
    
            optimizer.zero_grad()
    
            # lstm model
            x3_local = x3_local.view(x3_local.shape[0], nb_inputs, 4)
            output = net(x3_local)
            
            loss = loss_fn(output, y_local)
            
            loss.backward()
            optimizer.step()
            loss_hist.append(loss.item())
            
        ber = test_teacher(test_loader, net, rc, nb_inputs, num_frame=19) 
        if ber < best_ber:
            best_ber = ber
            #torch.save(net.state_dict(), './teacher.pt')
    
    return best_ber

def train_mlp(trainloader, testloader, test_teacher, net, rc, nb_inputs, lr=2e-3, nb_epochs=10):
    optimizer = torch.optim.Adam(net.parameters(), lr=lr, betas=(0.9,0.999), weight_decay=1e-4)
    scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=30, gamma=0.1)
    
    loss_fn = torch.nn.L1Loss()
    loss_hist = []
    best_ber = 10000000000.0
    
    for e in range(nb_epochs):
        loss_hist = []
        for x1_local, x2_local, x3_local, y_local in trainloader:
            x1_local = x1_local.float().to(device)
            x2_local = x2_local.float().to(device)
            x3_local = x3_local.float().to(device)
            y_local = y_local.float().to(device)
    
            optimizer.zero_grad()
    
            # lstm model
            x3_local = x3_local.view(x3_local.shape[0], nb_inputs * 4)
            output = net(x3_local)
            
            loss = loss_fn(output, y_local)
            
            loss.backward()
            optimizer.step()
            loss_hist.append(loss.item())
            
        ber = test_teacher(test_loader, net, rc, nb_inputs, num_frame=19) 
        if ber < best_ber:
            best_ber = ber
            #torch.save(net.state_dict(), './teacher.pt')
    
    return best_ber

In [7]:
silent = True
method = 'RLS'  # RLS; INV; INV+RLS
# N_total_frame = 17
N_total_frame = 94
N_sync_frame = 4
# SNR_list = np.arange(1,20,2)
SNR_list = [15]

# Dataset selection
folder_name = 'data/S2/'  # LOS_Near:S2, LOS_Far:S3, NLOS:S1
output_folder = 'data_outputs/S1'

if folder_name == 'data/S1/':  # NLOS
    delay = 0
    packet_num = 21
elif folder_name == 'data/S2/':  # LOS_Near
    delay = 1
    packet_num = 27 # correct
elif folder_name == 'data/S3/':  # LOS_Far
    delay = 1
    packet_num = 22 # 23
else:
    print("Undefined Dataset")
    exit(1)
    
window_size = 2
N_reservoir = 16
debug = False

ber_record = []
dfe_ber_record = []
LS_ber_record = []
comb_ber_record = []
sta_ber_record = []
tanh_lut = tanh(
    input_bit=8,
    dx_bit=8,
    slope_fmt=(10, 10),
    intercept_fmt=(19, 19),
    max=8,
    better_lut=True,
    verbose=False,
    plot=False)

SNR = SNR_list[0]

overall_rnn_ber = 0.0
overall_cnn_ber = 0.0
overall_mlp_ber = 0.0
cnt = 0

# except 2
packets = [11, 13, 14, 16, 17, 18, 19, 20]
for i in packets:
    print(i)
    rc = RC(silent, method, N_total_frame, N_sync_frame, SNR, delay, window_size, i,
            N_reservoir=16,
            spectral_radius=0.2,
            sparsity=0.4,
            noise=1e-6,
            lut_activation=False,  # True,
            tanh_lut=tanh_lut,
            input_scale=25,  #40, #50, # 25,
            reservoir_input_scale = 8,  #4,  #5,
            show_wout=False,
            output_folder= output_folder,
            debug=debug,
            use_fpga= None,
            sock= None,  # usock
            addr = None) # addr

    train_input, train_label, test_input, test_label = rc.run()
    #RC_test_input = np.load('gt_test_input_1.npy')
    #RC_train_input = np.load('gt_train_input_1.npy')
    #RC_train_label = np.load('gt_train_label_1.npy')
 
    train_mean = np.mean(train_input)
    train_std = np.std(train_input)

    train_input = (train_input - train_mean) / train_std
    test_input = (test_input - train_mean) / train_std
    train_label = 1.0 * train_label
    
    train_input, train_label, test_input, test_label = pre_processing(train_input, train_label)
    
    
    nb_inputs  = 6
    nb_steps = 100
    batch_size = 128

    import scipy.io

    from dataset import Dataset, RegTorchSeasonalitySpikingDataset, RegSpikingDataset, RegTorchSpikingDataset, RegTorchSeasonalityLinearSpikingDataset
    train_data = RegTorchSeasonalityLinearSpikingDataset(train_input, train_label, nb_inputs, nb_steps)
    test_data = RegTorchSeasonalityLinearSpikingDataset(test_input, test_label, nb_inputs, nb_steps)
    train_loader = DataLoader(dataset=train_data, batch_size=batch_size, shuffle=False, drop_last=False)
    test_loader = DataLoader(dataset=test_data, batch_size=batch_size, shuffle=False, drop_last=False)
    

    # RNN model 
    net_lstm = LSTMNet(4, 2).to(device)
    net_cnn = CNNNet(nb_inputs, 4, 2).to(device)
    net_mlp = MLPNet(nb_inputs * 4, 2).to(device)
    
    rnn_best_ber = train_dnn(train_loader, test_loader, test_rnn, net_lstm, rc, nb_inputs, lr=1e-3, nb_epochs=100)
    cnn_best_ber = train_dnn(train_loader, test_loader, test_cnn, net_cnn, rc, nb_inputs, lr=1e-3, nb_epochs=100)
    mlp_best_ber = train_mlp(train_loader, test_loader, test_mlp, net_mlp, rc, nb_inputs, lr=1e-3, nb_epochs=100)
    overall_rnn_ber += rnn_best_ber
    overall_cnn_ber += cnn_best_ber
    overall_mlp_ber += mlp_best_ber
    print(rnn_best_ber)
    print(cnn_best_ber)
    print(mlp_best_ber)
    cnt += 1

print("rnn ber: ", overall_rnn_ber / len(packets))
print("cnn ber: ", overall_cnn_ber / len(packets))
print("mlp ber: ", overall_mlp_ber / len(packets))

11
0.11513157894736842
0.11348684210526316
0.09923245614035088
13
0.054276315789473686
0.049890350877192985
0.05921052631578947
14
0.049342105263157895
0.03673245614035088
0.04550438596491228
16
0.11403508771929824
0.12116228070175439
0.11403508771929824
17
0.08333333333333333
0.0805921052631579
0.0581140350877193
18
0.015350877192982455
0.023574561403508772
0.05043859649122807
19
0.023574561403508772
0.027412280701754384
0.03070175438596491
20
0.05866228070175439
0.03728070175438596
0.044407894736842105
rnn ber:  0.06421326754385964
cnn ber:  0.06126644736842106
mlp ber:  0.06270559210526316


rnn ber:  0.06421326754385964
cnn ber:  0.06126644736842106
mlp ber:  0.06270559210526316

0 SNR
rnn ber:  0.17763157894736842
cnn ber:  0.18277138157894735
mlp ber:  0.18290844298245612


5 SNR
rnn ber:  0.0995751096491228
cnn ber:  0.11437774122807018
mlp ber:  0.1186951754385965

10 SNR
rnn ber:  0.06428179824561403
cnn ber:  0.07072368421052631
mlp ber:  0.08216831140350878

rnn ber:  0.06777686403508772
cnn ber:  0.07819353070175439
mlp ber:  0.0918311403508772


15 SNR
rnn ber:  0.0414610745614035
cnn ber:  0.05975877192982456
mlp ber:  0.06400767543859649

rnn ber:  0.05921052631578948
cnn ber:  0.05777138157894737
mlp ber:  0.06928453947368422

# power 
RNN = 43W   time = 0.0002124090003967285   energy = 0.0091335870170593255
CNN = 43W   time = 0.000151626079082489    energy = 0.006519921400547027
MLP = 43W   time = 0.00010849969625473023  energy = 0.004665486928

SNN    energy=0.00189